# Technical Factor - Accumulation Ratio & Buying Ratio (ARBR)

In [1]:
import dai
import pandas as pd

In [2]:
sd = '2025-01-01'
ed = '2026-01-01'

instrument_list = ['600519.SH']

In [3]:
sql = """
WITH
data_base AS (
    SELECT
        date,
        instrument,
        open,
        high,
        low,
        close,
        volume,
    FROM cn_stock_bar1d
),

data_factor AS (
    SELECT
        date,
        instrument,
        m_sum(high - open, 26) / m_sum(open - low, 26) AS AR,
        m_sum(high - m_lag(close, 1), 26) / m_sum(m_lag(close, 1) - low, 26) AS BR,
        AR / BR AS ARBR,
    FROM data_base
),

data_signal_trend AS (
    SELECT
        date,
        instrument,
        IF(ARBR > 1 AND m_lag(ARBR, 1) <= 1, 1, 0) AS TRBY1,
        IF(ARBR < 1 AND m_lag(ARBR, 1) >= 1, 1, 0) AS TRSL1,
    FROM data_factor
),

data_signal_momentum AS (
    SELECT
        date,
        instrument,
        IF(ARBR > 1 AND (ARBR - m_lag(ARBR, 1)) > 0, 1, 0) AS MTBY1,
        IF(ARBR < 1 AND (ARBR - m_lag(ARBR, 1)) < 0, 1, 0) AS MTSL1,
    FROM data_factor
),

data_signal_reversal AS (
    SELECT
        date,
        instrument,
        IF(ARBR < 0.7 AND (ARBR - m_lag(ARBR, 1)) > 0, 1, 0) AS RVBY1,
        IF(ARBR > 1.5 AND (ARBR - m_lag(ARBR, 1)) < 0, 1, 0) AS RVSL1,
    FROM data_factor
),

data_signal_breakout AS (
    SELECT
        date,
        instrument,
        IF(m_lag(ARBR, 1) <= 1.2 AND ARBR > 1.2, 1, 0) AS BKBY1,
        IF(m_lag(ARBR, 1) >= 0.8 AND ARBR < 0.8, 1, 0) AS BKSL1,
    FROM data_factor
),

data_combined AS (
    SELECT
        date,
        instrument,
        AR,
        BR,
        ARBR,
        TRBY1,
        TRSL1,
        MTBY1,
        MTSL1,
        RVBY1,
        RVSL1,
        BKBY1,
        BKSL1,
    FROM data_factor
    JOIN data_signal_trend    USING (date, instrument)
    JOIN data_signal_momentum USING (date, instrument)
    JOIN data_signal_reversal USING (date, instrument)
    JOIN data_signal_breakout USING (date, instrument)
)

SELECT *
FROM data_combined
QUALIFY COLUMNS(*) IS NOT NULL
ORDER BY date, instrument
"""

In [4]:
df = dai.query(sql, filters={"date":[sd,ed], "instrument":instrument_list}).df()
df

,date,instrument,AR,BR,ARBR,TRBY1,TRSL1,MTBY1,MTSL1,RVBY1,RVSL1,BKBY1,BKSL1
0,2025-02-17,600519.SH,1.055844,1.161708,0.908872,0,0,0,0,0,0,0,0
1,2025-02-18,600519.SH,1.198076,1.255300,0.954414,0,0,0,0,0,0,0,0
2,2025-02-19,600519.SH,1.272272,1.528653,0.832283,0,0,0,1,0,0,0,0
3,2025-02-20,600519.SH,1.261223,1.399101,0.901453,0,0,0,0,0,0,0,0
4,2025-02-21,600519.SH,1.305615,1.506182,0.866838,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
212,2025-12-25,600519.SH,0.573598,0.977376,0.586875,0,0,0,1,0,0,0,0
213,2025-12-26,600519.SH,0.568279,0.966250,0.588128,0,0,0,0,1,0,0,0
214,2025-12-29,600519.SH,0.531337,0.888712,0.597872,0,0,0,0,1,0,0,0
215,2025-12-30,600519.SH,0.519792,0.861970,0.603028,0,0,0,0,1,0,0,0
